## Import required modules

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf

## Download historical data of the stock from Yahoo Finance

In [ ]:
start_date = dt.datetime(2000, 1, 1)
end_date = dt.datetime(2024, 8, 31)

stock_data = yf.download("AAPL", start = start_date, end = end_date)
stock_data.head()

## Calculate stock's volatility using historical prices

In [ ]:
stock_data["Daily Return"] = stock_data["Adj Close"].pct_change()
daily_sd = stock_data["Daily Return"].std()
annualized_sd = np.sqrt(252) * daily_sd
annualized_sd

## Binomial Option Pricing Model for European Call/Put Options

In [ ]:
s0 = 229               # Stock Price
k = 200            # Strike Price
t = 1                  # Time to maturity (years)
rf = 0.039              # Annual risk-free rate
n = 4                  # Number of periods
sigma = annualized_sd  # Stock's Volatility
option_type = "c"      # Type of the option Call/Put

In [ ]:
def binomial_tree_c_pricing(s0, k, t, rf, n, sigma, option_type):
    dt = t/n
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    p = (np.exp(rf*dt) - d) / (u-d)
    discount_rate = np.exp(-rf * dt)

    stock_prices = np.zeros(n+1) 
    stock_prices[0] = s0 * (d**n)
    for j in range(1, n+1):
        stock_prices[j] = stock_prices[j-1] * u/d

    option_values = np.zeros(n+1)
    for j in range(0, n+1):
        if option_type == "c":
            option_values[j] = max(0, stock_prices[j] - k)
        else:
            option_values[j] = max(0, k - stock_prices[j])

    for i in np.arange(n, 0, -1):
        for j in range(0, i):
            option_values[j] = (p * option_values[j+1] + (1-p) * option_values[j]) * discount_rate

    return option_values[0]

binomial_tree_c_pricing(s0, k, t, rf, n, sigma, option_type)